In [ ]:
import sys, os
pbin_dir = os.path.dirname(sys.executable)
%env PBIN_DIR={pbin_dir}

In [ ]:
!${PBIN_DIR}/pip install gensim
import json
import pydash
from torchtext.data import get_tokenizer
from gensim.models import word2vec
tokenizer = get_tokenizer("basic_english")
#with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_politics_minill3_train.json') as f:
#with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_nfl_minill3_train.json') as f:
with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_nfl_minill3_train.json') as f:
#with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_politics_minill3_train.json') as f:
    jsond = json.loads(f.read())
sentences = []
for author in jsond:
    hists = jsond[author]
    for hist in hists:
        for key in hist:
            if key.startswith('context/'):
                sentences.append(hist[key])
            sentences.append(hist['Context'])
            sentences.append(hist['Response'])
sentences = pydash.uniq_by(sentences)
wordss = [tokenizer(sentence) for sentence in sentences]
#padlen = 50
#for i, words in enumerate(wordss):
#    wordss[i] = ['<PAD>'] * (padlen - len(words)) + words
print(wordss[0])


In [ ]:
import multiprocessing
#https://radimrehurek.com/gensim/models/word2vec.html#
#https://rare-technologies.com/word2vec-tutorial/
model = word2vec.Word2Vec(wordss,vector_size=200,workers=multiprocessing.cpu_count())


In [ ]:
wv = model.wv
vocab = {'<PAD>':0}
#PADは本家でも0になってる
embeddings = [[0.0]*200]
print(len(wv))
for i in range(len(wv)):
    vocab[wv.index_to_key[i]] = i+1
    #normalizeして出力する方法もあるがcikmのembeddingみるとしてなさそう
    embeddings.append(wv[i])
import pickle
with open("vocab_and_embeddings_rednfl.pkl", mode="wb") as f:
    pickle.dump([vocab,embeddings], f)


In [ ]:
#!mv vocab_and_embeddings_rednfl.pkl vocab_and_embeddings_redpol.pkl
!mv vocab_and_embeddings_rednfl.pkl vocab_and_embeddings_nfl_minill3.pkl
#!mv vocab_and_embeddings_rednfl.pkl vocab_and_embeddings_politics_minill3.pkl
!pwd
print("done.")